In [1]:
import requests
import json 
import plotly.express as px
import streamlit as st
import pandas as pd
import numpy as np

** Rouen Parking API**

+ Using this API to get parking informaiton : https://data.strasbourg.eu/explore/dataset/parkings/table/
+ Using this API to get real time information https://data.opendatasoft.com/explore/dataset/occupation-parkings-temps-reel%40eurometrostrasbourg/api/: 

In [2]:
Parking_info = requests.get("https://data.strasbourg.eu/api/explore/v2.1/catalog/datasets/parkings/records?limit=100&timezone=europe%2FParis") 
Parking_info

<Response [200]>

In [3]:
Dump_parking_info = json.dumps(Parking_info.json())
Loads_parking_info = json.loads(Dump_parking_info)  
parking_info = pd.json_normalize(Loads_parking_info['results'])
parking_info.head(2)

,city,friendlyurl,districtcode,description,address,types,name,idsurfs,zipcode,citycode,...,accessforelder,street,accessforwheelchair,accessfordeaf,normalizedalias,accessforblind,schedulelinkurl,hasurlschedule,position.lon,position.lat
0,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/452...,SQ_116,Afin de faciliter le stationnement de votre vé...,4 Rue Catherine Kany 67000 Strasbourg France,Cat_22_02,Parking Rhénapark,2471_DEP_64,67000,C_482,...,0,4 Rue Catherine Kany,1,0,parking-rhenapark,0,{},0,7.792645,48.570974
1,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/452...,SQ_112,Parking moyenne durée Reservation et location ...,4 rue Edmond Michelet 67000 Strasbourg France,Cat_22_02,Parking Danube Bleu,2472_DEP_65,67000,C_482,...,0,4 rue Edmond Michelet,1,0,parking-danube-bleu,0,{},0,7.766533,48.572834


**Real Time information**

In [4]:
Parking_reel_tine = requests.get("https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/occupation-parkings-temps-reel@eurometrostrasbourg/records?limit=100&timezone=europe%2FParis") 
Parking_reel_tine

<Response [200]>

In [25]:
Dump_RT_parking = json.dumps(Parking_reel_tine.json())
Loads_RT_parking = json.loads(Dump_RT_parking)  
RT_parking = pd.json_normalize(Loads_RT_parking['results'])

#Compute data
RT_parking["Occupee"] = RT_parking["total"]  - RT_parking["libre"] 
RT_parking["percentage_occupe"] = RT_parking["Occupee"]  / RT_parking["total"]  

RT_parking.head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,Occupee,percentage_occupe
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,175,170,ORANGE,175,780,0.816754
1,Parking Zénith,1400_DEP_26,1,2,Fermé,3175,3148,0,BLACK,Fermé,27,0.008504
2,Parking Centre Opéra Broglie,1378_DEP_4,3,1,Ouvert,447,90,90,GREEN,90,357,0.798658


In [26]:
#import package
import plotly.io as pio

#And the following will do the trick:
pio.templates.default = "plotly"

In [40]:
fig = px.bar(RT_parking, x='nom_parking', y='Occupee',color="realtimestatus")
fig.update_layout(xaxis={'categoryorder':'total descending'}) 
fig.show()


c:\Users\mccabedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

